this is a developer notebook for testing out red cell detection. 

If you find this notebook somewhere, ignore it! There are suite3d demos labeled "Demo-....." that are the official demos.


In [1]:
import os 
from datetime import date
from matplotlib import pyplot as plt
import numpy as np

# we need to set the current path to the directory
# containing the suite3d repository, this hack should
# do the trick
os.chdir(os.path.dirname(os.path.abspath("")))

from suite3d.job import Job
from suite3d import io
from suite3d import plot_utils as plot

Install gitpython for dev benchmarking to work


In [3]:
# update this to point to the demo data! 

tifs = io.get_tif_paths(r"D:\localData\suite3d\tinya_test_data")
for tif in tifs: print(tif)

D:\localData\suite3d\tinya_test_data\2025-02-24_3_TC030_2P_00001_00001.tif
D:\localData\suite3d\tinya_test_data\2025-02-24_3_TC030_2P_00001_00002.tif
D:\localData\suite3d\tinya_test_data\2025-02-24_3_TC030_2P_00001_00003.tif
D:\localData\suite3d\tinya_test_data\2025-02-24_3_TC030_2P_00001_00004.tif
D:\localData\suite3d\tinya_test_data\2025-02-24_3_TC030_2P_00001_00005.tif
D:\localData\suite3d\tinya_test_data\2025-02-24_3_TC030_2P_00001_00006.tif
D:\localData\suite3d\tinya_test_data\2025-02-24_3_TC030_2P_00001_00007.tif
D:\localData\suite3d\tinya_test_data\2025-02-24_3_TC030_2P_00001_00008.tif
D:\localData\suite3d\tinya_test_data\2025-02-24_3_TC030_2P_00001_00009.tif
D:\localData\suite3d\tinya_test_data\2025-02-24_3_TC030_2P_00001_00010.tif


In [4]:
# Set the mandatory parameters
params = {
    # volume rate
    'fs': io.get_vol_rate(tifs[0]),
    
    # planes to analyze. 0 is typically the flyback, so we exclude it here
    'planes' : np.array([1,2,3]), 
    # number of planes recorded by scanimage, including the flyback
    'n_ch_tif' : 9,
    
    # Decay time of the Ca indicator in seconds. 1.3 for GCaMP6s. This example is for GCamP8m
    'tau' : 1.3,
    'lbm' : False, 
    'num_colors' : 2, # how many color channels were recorded by scanimage
    'functional_color_channel' : 0, # which color channel is the functional one
     # voxel size in z,y,x in microns
    'voxel_size_um' : (20, 1.5, 1.5),

    # number of files to use for the initial pass
    # usually, ~500 frames is a good rule of thumb
    # we will just use 200 here for speed
    'n_init_files' :  1,

    # 3D GPU registration - fast! 
    '3d_reg' : True,
    'gpu_reg' : True,
    
    # note : 3D CPU is not supported yet
    'subtract_crosstalk' : False, # turn off some lbm-only features
    'fuse_strips' : False, # turn off some lbm-only features    
}

In [5]:
# Create the job
job = Job(r'D:\localData\suite3d\runs','test-redcell', tifs = tifs,
          params=params, create=True, overwrite=True, verbosity = 3)

Loading job directory for test-redcell in D:\localData\suite3d\runs
      Created dir D:\localData\suite3d\runs\s3d-test-redcell\registered_fused_data
      Created dir D:\localData\suite3d\runs\s3d-test-redcell\summary
      Created dir D:\localData\suite3d\runs\s3d-test-redcell\iters
   Loading default params
      Updating param fs
      Updating param planes
      Updating param n_ch_tif
      Updating param tau
      Updating param lbm
      Updating param num_colors
      Updating param functional_color_channel
      Updating param voxel_size_um
      Updating param n_init_files
      Updating param 3d_reg
      Updating param gpu_reg
      Updating param subtract_crosstalk
      Updating param fuse_strips
   Updated main params file


In [22]:
job.run_init_pass()

In [9]:
# If you have large tiffs, split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [11]:
# OPTIONAL: load and take a look at the reference image
summary = job.load_summary()
ref_img = summary['ref_img_3d']

# # view 1 plane at a time
# plot.show_img(ref_img[3], figsize=(3,4))

# # interactive 3D viewer
# plot.VolumeViewer(ref_img)


In [21]:
job.register()

In [20]:
corr_map = job.calculate_corr_map()

In [14]:
res = job.load_corr_map_results()
vmap = res['vmap']

In [18]:
job.params['patch_size_xy'] = (550, 550)
# for speed, only segment a single patch
job.segment_rois()

In [19]:
job.compute_npil_masks()
traces = job.extract_and_deconvolve()

   Updated main params file
   Movie shape: (5, 4000, 514, 513)
1736
   Extracting 1736 valid cells, and saving cell flags to /mnt/md0/runs/s3d-demo-std/rois/iscell_extracted.npy
   Extracting activity
         Will extract in 8 batches of 500
   Saving intermediate results to /mnt/md0/runs/s3d-demo-std/rois
   Deconvolving
   Saving to /mnt/md0/runs/s3d-demo-std/rois


In [ ]:
job.export_results('path/to/output',result_dir_name='rois')

To take a look at the outputs in napari, navigate to the suite3d directory in a command shell and run the following:
```
python curation.py curation --output_dir /path/to/output/rois
```